# ML on Forwards

In [5]:
relevant_forward_keys = ['togga_score', 'apps', 'tackle_total', 'yellow_cards', 'dispossessed_total', 'red_cards', 
                         'minutes', 'successful_dribbles', 'own_goals', 'interception_total', 
                         'goals_scored', 'key_passes', 'accurateCrosses_total', 'aerialWon_total', 'assists', 
                         'points_above_replacement', 'name', 'age', 'ontarget_scoring_att', 'next_year']

relevant_midfield_keys = ['togga_score', 'apps', 'tackle_total', 'yellow_cards', 'dispossessed_total', 'red_cards', 
                          'minutes', 'successful_dribbles', 'own_goals', 'interception_total', 'ontarget_scoring_att',
                          'goals_scored', 'key_passes', 'accurateCrosses_total', 'clean_sheets', 'aerialWon_total', 
                          'assists', 'points_above_replacement', 'name', 'age', 'next_year']

relevant_defense_keys = ['goals_conceded', 'togga_score', 'apps', 'tackle_total', 'yellow_cards', 'dispossessed_total',
                         'red_cards', 'minutes', 'successful_dribbles', 'own_goals', 'name', 'ontarget_scoring_att',
                         'interception_total', 'goals_scored', 'key_passes', 'accurateCrosses_total', 'clean_sheets', 
                         'aerialWon_total', 'assists', 'points_above_replacement', 'clearance_total', 'age', 'next_year']

relevant_goalie_keys = ['goals_conceded', 'saves', 'togga_score', 'apps', 'tackle_total', 'yellow_cards', 'red_cards', 
                        'minutes', 'own_goals', 'interception_total', 'key_passes', 'clean_sheets', 'aerialWon_total', 
                        'penalties_saved', 'points_above_replacement', 'name', 'clearance_total', 'age', 'next_year']

In [6]:
import json
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [7]:
with open("Data/positional.json", "r") as infile:
    data = json.load(infile)

#We will do analysis on Forwards
position = '4'
players = []
for player in data[position]:
    players.append(pd.DataFrame([player['history']['2015/16']]))
    players.append(pd.DataFrame([player['history']['2014/15']]))
    players.append(pd.DataFrame([player['history']['2013/14']]))
    players.append(pd.DataFrame([player['history']['2012/13']]))

players = pd.concat(players)
filter = players['togga_score'] > 0
players = players[filter]
keys = relevant_forward_keys.copy()
keys.remove('points_above_replacement')
players = players[keys]
players = players.dropna(axis=1,how='all')
players = players.dropna(axis=0,how='any')

players = players.set_index("name")
players.index.name = None
keys.remove('name')
    

In [8]:
import matplotlib.pyplot as plt
# box and whisker plots of some of the stats
plot_keys = ['togga_score', 'goals_scored', 'assists', 'ontarget_scoring_att', 'aerialWon_total', 'minutes', 
             'successful_dribbles', 'age', 'next_year']
players[plot_keys].plot(kind='box', subplots=True, layout=(3,3), sharex=False, sharey=False)
plt.show()

In [9]:
# histograms
players[plot_keys].hist()
plt.show()

## Linear Regressor

In [10]:
x_keys = keys.copy()
x_keys.remove('next_year')
X = players[x_keys]
Y = players['next_year']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=324)
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
Y_prediction = regressor.predict(X_test)
#X_test, y_prediction

RMSE = sqrt(mean_squared_error(y_true = Y_test, y_pred = Y_prediction))
print(RMSE)

100.95872239373855


## Decision Tree Regressor

In [11]:
regressor = DecisionTreeRegressor(max_depth=20)
regressor.fit(X_train, Y_train)
y_prediction = regressor.predict(X_test)
#y_prediction
#X_train.head()

RMSE = sqrt(mean_squared_error(y_true = Y_test, y_pred = y_prediction))
print(RMSE)

194.10167009413735


## Classification ML

In [12]:
from pandas.tools.plotting import scatter_matrix
from sklearn import cross_validation
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [13]:
players["category"] = players["togga_score"]
for index, row in players.iterrows():
    if row['next_year'] > 0:
        ratio = row['togga_score'] / abs(row['next_year'])
        if  ratio < 0.8:
            val = -1
        elif ratio < 1.2:
            val = 0
        else:
            val = 1
        #print(ratio,val)
    else:
        #row[x_keys] = 0
        #forwards.drop(index, inplace=True)
        val = -10
    players.set_value(index,'category',val)

    
x_keys = keys.copy()
x_keys.remove('next_year')
x_keys.append("category")
filter = players['next_year'] > 0
players = players[filter]
array = players[x_keys].values
X = array[:,:-1]
Y = array[:,-1].astype(int)
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = cross_validation.train_test_split(X, Y, test_size=validation_size, random_state=seed)

seed = 7
scoring = 'accuracy'

models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = cross_validation.KFold(len(X_train), n_folds=10, random_state=seed)
    cv_results = cross_validation.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.453333 (0.255256)


C:\Users\Noah\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\Noah\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\Noah\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\Noah\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\Noah\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\Noah\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\Noah\Anaconda3\lib\

LDA: 0.560000 (0.129786)
KNN: 0.346667 (0.256991)
CART: 0.600000 (0.260342)
NB: 0.633333 (0.244949)
SVM: 0.516667 (0.273354)


In [14]:
# Compare Algorithms
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

# Predictions

In [15]:
new_data = []
for player in data[position]:
    new_data.append(pd.DataFrame([player['history']['2016/17']]))

new_data = pd.concat(new_data)
filter = new_data['togga_score'] > 0
new_data = new_data[filter]
new_data = new_data.dropna(axis=1,how='all')
new_data = new_data.dropna(axis=0,how='any')

All_predictions = []
All_predictions.append(new_data['name'].values.astype('str'))
new_data = new_data.set_index("name")
new_data.index.name = None
new_keys = keys.copy()
new_keys.remove('next_year')
new_X = new_data[new_keys].values


# Make predictions on validation dataset
def makePredictions(model):
    model.fit(X_train, Y_train)
    predictions = model.predict(X_validation)
    print(accuracy_score(Y_validation, predictions))
    print(confusion_matrix(Y_validation, predictions))
    print(classification_report(Y_validation, predictions))
    predictions = model.predict(new_X)
    #for name, pred in zip(new_data.index.values, predictions):
        #print(name,pred)
    All_predictions.append(predictions)

## Decision Tree Predictions

In [16]:
makePredictions(DecisionTreeClassifier())

0.533333333333
[[1 0 0]
 [2 1 0]
 [3 2 6]]
             precision    recall  f1-score   support

         -1       0.17      1.00      0.29         1
          0       0.33      0.33      0.33         3
          1       1.00      0.55      0.71        11

avg / total       0.81      0.53      0.60        15



## LDA Predictions

In [17]:
makePredictions(LinearDiscriminantAnalysis())

0.466666666667
[[1 0 0]
 [1 2 0]
 [7 0 4]]
             precision    recall  f1-score   support

         -1       0.11      1.00      0.20         1
          0       1.00      0.67      0.80         3
          1       1.00      0.36      0.53        11

avg / total       0.94      0.47      0.56        15



C:\Users\Noah\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


## Gaussian NB Predictions

In [18]:
makePredictions(GaussianNB())

0.6
[[1 0 0]
 [1 1 1]
 [2 2 7]]
             precision    recall  f1-score   support

         -1       0.25      1.00      0.40         1
          0       0.33      0.33      0.33         3
          1       0.88      0.64      0.74        11

avg / total       0.72      0.60      0.63        15



## Logistic Regression Predictions

In [19]:
makePredictions(LogisticRegression())

0.6
[[0 0 1]
 [1 1 1]
 [3 0 8]]
             precision    recall  f1-score   support

         -1       0.00      0.00      0.00         1
          0       1.00      0.33      0.50         3
          1       0.80      0.73      0.76        11

avg / total       0.79      0.60      0.66        15



## KNN Predictions

In [20]:
makePredictions(KNeighborsClassifier())

0.533333333333
[[0 1 0]
 [1 2 0]
 [2 3 6]]
             precision    recall  f1-score   support

         -1       0.00      0.00      0.00         1
          0       0.33      0.67      0.44         3
          1       1.00      0.55      0.71        11

avg / total       0.80      0.53      0.61        15



## SVC Prediction

In [21]:
makePredictions(SVC())

0.733333333333
[[ 0  0  1]
 [ 0  0  3]
 [ 0  0 11]]
             precision    recall  f1-score   support

         -1       0.00      0.00      0.00         1
          0       0.00      0.00      0.00         3
          1       0.73      1.00      0.85        11

avg / total       0.54      0.73      0.62        15



C:\Users\Noah\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [22]:
print(All_predictions)

[array(['Olivier Giroud', 'Callum Wilson', 'Benik Afobe', 'Sam Vokes',
       'Andre Gray', 'Ashley Barnes', 'Michy Batshuayi',
       'Diego Da Silva Costa', 'Romelu Lukaku', 'Abel Hernández',
       'Jamie Vardy', 'Shinji Okazaki', 'Christian Benteke',
       'Daniel Sturridge', 'Divock Origi', 'Sergio Agüero', 'Wayne Rooney',
       'Marcus Rashford', 'Zlatan Ibrahimovic', 'Cristhian Stuani',
       'Shane Long', 'Peter Crouch', 'Fabio Borini', 'Jermain Defoe',
       'Harry Kane', 'Vincent Janssen', 'Troy Deeney', 'Salomón Rondón',
       'Enner Valencia', 'Álvaro Negredo', 'Fernando Llorente',
       'Stefano Okaka', 'Islam Slimani'], 
      dtype='<U20'), array([ 1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
       -1,  0,  1,  1,  0,  1,  0,  0,  1, -1,  1, -1,  1,  1,  1,  1]), array([ 1,  1,  1,  0,  1,  1,  1,  0,  0,  1,  0,  1,  1,  1,  0, -1,  1,
        1,  1,  1,  1,  0,  1,  1,  1, -1,  0,  0,  1,  1,  1,  1,  1]), array([ 1,  1,  1,  1,  1,  1,  1,

In [23]:
from itertools import chain
chain = chain.from_iterable(zip(*All_predictions))
preds = list(chain)
all_preds = 'Name\tCART\tLDA\tGNB\tLR\tKNN\tSVC\n'
for i in range(int(len(preds)/7)):
    #print(preds[i*7], preds[i*7+1:i*7+7])
    all_preds+=(preds[i*7])
    for j in range(i*7+1, i*7+7):
        all_preds+='\t' + str(preds[j])
    all_preds+='\n'
#print(all_preds) 
all_preds.encode("utf-8")
with open("Data/Predictions/Forward.txt", "w") as f:
    f.write(all_preds)

In [24]:
import csv

txt_file = r"Data/Predictions/Forward.txt"
csv_file = r"Data/Predictions/Forward.csv"

in_txt = csv.reader(open(txt_file, "r"), delimiter = '\t')
out_csv = csv.writer(open(csv_file, 'w', newline=''))
out_csv.writerows(in_txt)
del out_csv
del in_txt